# 在同一网格上可视化Metop和MSG数据## 摘要在本教程中，我们将探讨如何将Metop和MSG数据都投影到同一网格上。
## 第一步，加载数据加载数据的方式与之前教程中展示的一样
## 练习 6.1：将数据对齐到同一网格创建一个用于Metop数据的`Scene`对象和另一个用于MSG数据的`Scene`对象。从MSG加载`day_microphysics` RGB合成数据，并从Metop加载10.8微米通道。将两者都重采样到东欧区域。

In [ ]:
from satpy import Scene, find_files_and_readers
from datetime import datetime

metop_files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex6',
                                     reader='avhrr_l1b_eps',
                                     start_time=datetime(2018, 10, 7 ,9, 25),
                                     end_time=datetime(2018, 10, 7 ,9, 30))
scn_metop = Scene(filenames=metop_files)
scn_metop.load([10.8])

In [ ]:
msg_files = find_files_and_readers(base_dir='/tcenas/scratch/pytroll/ex6',
                                   reader='seviri_l1b_native')

scn = Scene(filenames=msg_files)
scn_msg = Scene(filenames=msg_files)
scn_msg.load(['day_microphysics','IR_108'])


In [ ]:
newscn_metop = scn_metop.resample('eurol')
newscn_metop.show(10.8)

In [ ]:

newscn_msg = scn_msg.resample('eurol')
newscn_msg.show('IR_108')


## 练习 6.2：图像处理要求：
- 保留 Markdown 格式
- 保留代码、网址、HTML 标签
- 使用标准技术术语
- 仅输出翻译，不加解释

要翻译的文本：
现在，我们希望将两个数据集绘制到同一图像上。

In [ ]:
from satpy.writers import get_enhanced_image
import xarray as xr
from trollimage.xrimage import XRImage

# Lets compare MSG and METOP channel 10.8 data
msg_image = get_enhanced_image(newscn_msg['IR_108'])

metop_image = get_enhanced_image(newscn_metop[10.8])

array1 = msg_image.data.where(metop_image.data.isnull(), metop_image.data)

XRImage(array1)




In [ ]:
# No Using MSG composite
msg_image = get_enhanced_image(newscn_msg['day_microphysics'])
color_array = xr.concat((metop_image.data, metop_image.data, metop_image.data), 'bands')
color_array['bands'] = ['R', 'G', 'B']
final_array = msg_image.data.where(color_array.isnull(), color_array.data)
XRImage(final_array)

## The `Multiscene` 类要求：
- 保留 Markdown 格式
- 保留代码、URL 和 HTML 标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

要翻译的文本：
为了同时操作多个场景，在某些情况下，利用 satpy 的 `Multiscene` 类可能会有优势。让我们创建一个 multiscene 实例，加载一些数据，并将数据重采样到 `EastEurope` 区域。

In [ ]:
from satpy import MultiScene
mscn = MultiScene([scn_msg, scn_metop])
mscn.load(['overview'])
new_mscn = mscn.resample('eurol')
blended_scene = new_mscn.blend()
blended_scene.show('overview', overlay={'coast_dir': '/tcenas/scratch/pytroll/shapes/', 'color': (255, 0, 0), 'resolution': 'i'})